The ArUco Markers used were generated in the following website:

https://chev.me/arucogen/

In [5]:
import cv2
import numpy as np
import os
import imageio

dist =[0.1639958233797625, -0.271840030972792, 0.001055841660100477, -0.00166555973740089, 0.0]
K = [322.0704122808738, 0.0, 199.2680620421962, 0.0, 320.8673986158544, 155.2533082600705, 0.0, 0.0, 1.0]
mtx = np.array(K).reshape(3, 3)
dist = np.array(dist).reshape(1,5)

#ArUco detector Initialization 

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100) 

#Do not change the tag used (cv2.aruco.DICT_4X4_100), only if we change the ArUcos we're using (and thus their tags) 

parameters =  cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

In [6]:
#In this section you can see if you're sucessfully reading an image 

image = imageio.v2.imread(os.path.abspath('Screenshot_3.png')) 
if image is not None:
    cv2.imshow('Image', image)
    cv2.waitKey(0)  # Wait for any key press
    cv2.destroyAllWindows()
else:
    print("Image not found or cannot be read")


In [93]:
#In this section you can apply the ArUco detector to the previous read image 

image = imageio.v2.imread(os.path.abspath('Screenshot_3.png'))
if image is not None:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, dictionary, parameters=parameters)
    if ids is not None and len(ids) > 0:
        for i in range(len(ids)):
            marker_corners = corners[i][0]

            cv2.polylines(image, [np.int32(marker_corners)], True, (0, 255, 0), 2) #Bounding Box
            rvec , tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.1, mtx, dist)
            
            #ID of the marker 
            cv2.putText(image, str(ids[i][0]), (int(marker_corners[0][0]-10),int(marker_corners[0][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            cv2.putText(image, str(round(tvec[0][0][2],5)), (int(marker_corners[2][0]-100),int(marker_corners[2][1])+60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('Image', image)
        cv2.waitKey(0)  # Wait for any key press
        cv2.destroyAllWindows()
    else:
        print("No ArUco markers detected in the image")

else:
    print("Image not found or cannot be read")

In [42]:
#In this section you can test the ArUco detector in real time 
x_axis = [1,0,0]
cap = cv2.VideoCapture(0)  
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, dictionary, parameters=parameters)

    if ids is not None and len(ids) > 0:
        for i in range(len(ids)):
            marker_corners = corners[i][0]
            
            cv2.polylines(frame, [np.int32(marker_corners)], True, (0, 255, 0), 2) #Bounding Box
            
            rvec , tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.1, mtx, dist)
                        
            axisPoints, _ = cv2.projectPoints(np.float32([[0,0,0], [1,0,0], [0,1,0], [0,0,1]]), rvec, tvec, mtx, dist)
        
            # Draw the axis lines
            #cv2.line(frame, tuple(axisPoints[0].astype(int).ravel()), tuple(axisPoints[1].astype(int).ravel()), (0,0,255), 3) # x-axis (red)
            #cv2.line(frame, tuple(axisPoints[0].astype(int).ravel()), tuple(axisPoints[2].astype(int).ravel()), (0,255,0), 3) # y-axis (green)
            #cv2.line(frame, tuple(axisPoints[0].astype(int).ravel()), tuple(axisPoints[3].astype(int).ravel()), (255,0,0), 3) # z-axis (blue)
            
            #Rotation relative to the marker
            Rot, _ = cv2.Rodrigues(rvec)
            x = Rot.dot(x_axis)
            theta = np.arccos(np.dot(np.transpose(x_axis),x)/(np.linalg.norm(x_axis)*np.linalg.norm(x)))
            
            #ID of the marker 
            cv2.putText(frame, str(ids[i][0]), (int(marker_corners[0][0]-10),int(marker_corners[0][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
            cv2.putText(frame, str(round(tvec[0][0][2],5)), (int(marker_corners[2][0]-80),int(marker_corners[2][1])+45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            cv2.putText(frame, 'ang='+str(round(theta,5)), (int(marker_corners[1][0]-70),int(marker_corners[1][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255 ), 3)
    cv2.imshow('Image', frame)

    if cv2.waitKey(1) != -1: #If any key is pressed
        cv2.destroyAllWindows()
        break
        
cap.release()